# Part 1 - Regularization Techniques

## Objective

This lab applies four major regularization techniques to the deep models built in Lab 3 to combat overfitting:

- **ℓ₁ and ℓ₂ Regularization:** Adding a penalty to the loss function based on the size of the weights.
- **Dropout:** Randomly "killing" neurons during training to ensure the network doesn't over-rely on specific paths.
- **Max-Norm Regularization:** Constraining the weights of each neuron so they don't grow too large.

## Conceptual Background

| Technique | Description | Effect |
|-----------|-------------|--------|
| **ℓ₁ Regularization** | Adds penalty proportional to absolute weight values | Produces sparse models (weights become exactly zero), effectively acting as feature selection |
| **ℓ₂ Regularization** | Adds penalty proportional to squared weight values | Keeps weights small but rarely zero, helps handle multicollinearity |
| **Dropout** | Randomly deactivates neurons during training | Forces the network to learn redundant representations, making it more robust |
| **Max-Norm** | Rescales weight vector w such that \|\|w\|\|₂ ≤ c | Prevents weights from growing too large, where c is the max-norm hyperparameter |

## Step 1: $\ell_1$ and $\ell_2$
## RegularizationWe will apply these to a Dense network similar to your MNIST/CIFAR-10 tasks.

In [1]:
import tensorflow as tf
from tensorflow import keras

# 1. Define l2 regularization factor
# This adds a penalty to the loss: Loss + 0.01 * sum(weights^2)
regularizer = keras.regularizers.l2(0.01)

model_l2 = keras.Sequential([
    keras.layers.Flatten(input_shape=[32, 32, 3]), # Using CIFAR-10 shape as example
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal",
                       kernel_regularizer=regularizer),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal",
                       kernel_regularizer=regularizer),
    keras.layers.Dense(10, activation="softmax")
])

# If you want to use l1, simply use: keras.regularizers.l1(0.01)
# For both (Elastic Net style): keras.regularizers.l1_l2(0.01, 0.01)

model_l2.compile(loss="categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])

2026-02-19 08:47:12.089219: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/clauds/anaconda3/envs/tf_env/lib/python3.12/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
